# 18.C06 Pset 1

A Julia (ipynb) version of this pdf file is available on the 18.C06 Canvas site in the Module labeled "Problem Set Julia files".

Due Friday 9/13 at **11am**; 10% penalty if it is turned in within 24 hours, and after than late psets will not be accepted.   Submit in PDF format: a decent-quality scan/image of any handwritten solutions (e.g. get a scanner app on your phone or use a tablet), and a PDF printout of your Jupyter notebook showing your code and (clearly labeled) results.

$\color{red}{\text{TO GENERATE A PDF OF YOUR JUPYTER NOTEBOOK, TRY THIS STRATEGY FIRST:}}$ From the File pull-down menu, select "Save and Export Note Book As", and then select the HTML format (IF YOU SELECT the PDF format, you will probably get error messages).  Then open the downloaded HTML file with your favorite browser, and use the browser's print function to generate the PDF file.


## Problem 1 (3+3+3 points)

Consider the linear system
$$
\begin{pmatrix} 1 & 2 \\ 3 & \alpha \end{pmatrix} \begin{pmatrix} x_1 \\ x_2 \end{pmatrix} = \begin{pmatrix} 5 \\ 15 \end{pmatrix}
$$

For what values of the parameter $\alpha$ (if any) does the linear system have:

**(a)** a unique solution

**(b)** no solutions

**(c)** infinitely many solutions

## Problem 2 (5+5 points)

(From Strang, section 2.2, problem 14.) Consider Gaussian elimination on the following system of equations:

\begin{align}
2x + 5y + z &= 0 \\
4x + dy + z &= 2 \\
y - z &= 3
\end{align}

(Write your solution in matrix form.)

* What number $d$ forces you to do a row exchange during elimination, and what (non-singular) triangular system do you obtain for that $d$?
* What value of $d$ would make this system singular (no third pivot, i.e. no way to get a triangular system with 3 nonzero values on the diagonal)?

## Problem 3 (5+5 points)

Suppose that we know 3 data points $(t_1, b_1), (t_2,b_2), (t_3,b_3)$ of an unknown function $b(t)$, and we wish to construct a smooth interpolating function $c(t)$ that passes through these three points.   One nice technique that generalizes very well to lots of data points, and even to higher dimensions, is an interpolating function of the form:
$$
c(t) = x_1 r(t - t_1) + x_2 r(t - t_2) + x_3 r(t - t_3)
$$
where $r(t)$ is some decaying function.  For example, $r(t)$ could be the Gaussian ("bell curve"):
$$
r(t) = e^{-t^2 / R^2}
$$
for some radius $R$.  That is, we are making a linear combination of Gaussians centered at each point $t_k$.

**(a)** If we require that $c(t)$ passes through all our data points (i.e. $c(t_k) = b_k$ for $k=1,2,3$), then you can write down a $3\times 3$ system of equations $Ax = b$ for the vector $x$ of our unknown coefficients:
$$
\underbrace{ \begin{pmatrix} r(t_1 - t_1) & ? & ? \\ ? & ? & ? \\ ? & ? & ? \end{pmatrix}}_A 
\underbrace{ \begin{pmatrix} x_1 \\ x_2 \\ x_3 \end{pmatrix}}_x = \underbrace{ \begin{pmatrix} b_1 \\ b_2 \\ b_3 \end{pmatrix}}_b
$$
What are the missing ("?") elements of $A$ (in terms of $r(t)$ and the $t_k$)?

**(b)**

Here are example values of $t$, $b$, and $R$ in Julia.  Fill in the missing ("?") elements of the matrix $A$.   Then, use `x = A \ b` to solve $Ax=b$ for $x$ and modify the code below to plot the interpolant $c(t)$.

One you fix the value of $x$ to the correct values, you should see that your interplant $c(t)$ **goes through the data points** (dots).

In [9]:
t = [1, 2, 4] # example data points t₁, t₂, t₃
b = [2, 1, 3] # example data points b₁, b₂, b₃
R = 2 # example "radius" of the basis function
r(t) = exp(-t^2 / R^2)

r (generic function with 1 method)

In [10]:

A = [ r(t[1]-t[1]) r(t[1]-t[2]) r(t[1]-t[3])
      r(t[2]-t[1]) r(t[2]-t[2]) r(t[2]-t[3])
      r(t[3]-t[1]) r(t[3]-t[2]) r(t[3]-t[3])]

3×3 Matrix{Float64}:
 1.0       0.778801  0.105399
 0.778801  1.0       0.367879
 0.105399  0.367879  1.0

In [11]:
x = A \ b # solve Ax=b for x

3-element Vector{Float64}:
  4.985428520853115
 -4.386657445791713
  4.088300789561882

In [12]:
using PyPlot

ts = range(0, 5, length=200) # 200 points from 0 to 5
c(T, x, t) = x[1] * r(T - t[1]) + x[2] * r(T - t[2]) + x[3] * r(T - t[3]) # interpolating function ∑ xᵢ r(T - tᵢ)

plot(t, b, "bo") # plot the input data
xlim(0,5)
ylim(0,4)
xlabel(L"t")
ylabel(L"b")
title("problem 3(b): interpolation")

# delete this line to use x = A \ b from above
# x = [4,-4,4] # WRONG coefficients

# plot the interpolant c(t)
plot(ts, c.(ts, (x,), (t,)), "b-", linewidth=2)

legend(["data points", "interpolant"])

PyObject <matplotlib.legend.Legend object at 0x137db0bc0>

## Problem 4 (3+3+3+3+3+3+3 points)

Given a function $f(x)$, we can *sample* it at $n$ equally spaced points $x_1 = a, x_2 = a+\Delta x, x_3 = a+2\Delta x, \ldots, x_n = a+(n-1)\Delta x = b$ from $a$ to $b$ to obtain a column vector
$$
\vec{f} = \begin{pmatrix} f(x_1) \\ f(x_2) \\ \vdots \\ f(x_n) \end{pmatrix} = \begin{pmatrix} f_1 \\ f_2 \\ \vdots \\ f_n \end{pmatrix} \, ,
$$
where $f_k = f(x_k) = f(a + (k-1)\Delta x)$.

**(a)** An *approximate derivative* of $f(x)$ (called a [finite-difference approximation](https://en.wikipedia.org/wiki/Finite_difference)) is given by the *difference* operation:
$$
D\vec{f} = \frac{1}{\Delta x} \begin{pmatrix} f_2 - f_1 \\ f_3 - f_2 \\ \vdots \\ f_n - f_{n-1} \end{pmatrix}
$$
Write down this linear operator $D$ as a matrix (what size?) for $n=5$.

**(b)** Construct the *same* $D$ matrix in Julia, for $\Delta x = 1$, using the [`diagm` function](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.diagm) to build matrices out of diagonal entries, and the [`fill` function](https://docs.julialang.org/en/v1/base/arrays/#Base.fill) to make vectors filled with constants.  For example, this is a $6 \times 8$ matrix with 1's, 4's, and 7's on three diagonals:

In [56]:
using LinearAlgebra
diagm(6,8, 0 => fill(1,6), 1 => fill(4,6), -2 => fill(7,4))

6×8 Matrix{Int64}:
 1  4  0  0  0  0  0  0
 0  1  4  0  0  0  0  0
 7  0  1  4  0  0  0  0
 0  7  0  1  4  0  0  0
 0  0  7  0  1  4  0  0
 0  0  0  7  0  1  4  0

In [ ]:
D = ??? # FILL IN YOUR CODE HERE

**(c)** Use your same code to construct a $D$ matrix for $n=100$ and apply it to approximately differentate $f(x) = \sin(x)$ for $a,b = 0,5$, and plot your result using the code below.  It should look like a cosine!

In [ ]:
a, b = 0, 5
n = 100
Δx = (b - a) / (n - 1) # spacing for n points from a to b

x = range(a, b, length=n) # array of n points from a to b
f = sin.(x) # apply sin(x) elementwise to the vector x

D = ??? # FILL IN YOUR CODE HERE

# plot f and D*f
using PyPlot
plot(x, f, "r-")
plot(x[1:end-1] .+ Δx/2, D*f, "b-")

title(L"$f(x) = \sin(x)$ and approximate derivative")
xlabel(L"x")
legend([L"f(x)", L"df/dx"])

**(d)** An *approximate integral* of a function $g(x)$ sampled at $m$ points with spacing $\Delta x$ is given by a *sum*:
$$
S \vec{g} = S \underbrace{\begin{pmatrix} g_1 \\ g_2 \\ \vdots \\ g_m \end{pmatrix}}_g = (g_1 + g_2 + \cdots g_m) \Delta x \, ,
$$
where $g_k = g(x_k)$ similar to $\vec{f}$.  Write the linear operator $S$ as a matrix for $m=6$.

**(e)** What is the product $S D$ for $n=6$ and $m = \_\_\_\_$ (for the same $\Delta x$)?  (That is, the approximate "integral of the derivative" is what linear operator?)

**(f)** Give a nonzero vector $\vec{f}$ such that $D\vec{f} = \vec{0}$ (for the $n = 5$ case).   In fact, there are infinitely many such vectors because … ?

**(g)** Your answer from part (f) implies that the last column of $D$ can be formed by what linear combination of the previous columns of $D$?